<a href="https://colab.research.google.com/github/Ethan-Chen-plus/PythonStudy/blob/main/Stable_Diffusion_WebUI_Colab_TW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion WebUI Colab TW

### 使用流程

1. 到**設置模型**及**進階設置**進行設置
2. 左上角工具列 > 執行階段 > 變更執行階段類型 > 硬體加速器改為 GPU
3. 左上角工具列 > 執行階段 > 全部執行
4. 到**啟動 WebUI** 點選 public URL

### 巴哈原文
https://home.gamer.com.tw/artwork.php?sn=5664550

### 參考資料
https://github.com/ddPn08/automatic1111-colab

## 1 - 前置步驟

In [ ]:
# @title 1.1 下載 stable-diffusion-webui
%cd /content/
repository_url = "https://github.com/AUTOMATIC1111/stable-diffusion-webui"  # @param {type: "string"}
webui_branch = "master"  # @param {type: "string"}

! git clone {repository_url}
%cd /content/stable-diffusion-webui
! git checkout {webui_branch}

In [ ]:
# @title 1.2 建立雲端資料夾
%cd /content
# @markdown 雲端資料夾名稱
data_dir_name = "Stable_Diffusion_WebUI_Colab_TW"  # @param {type:"string"}
data_dir = f"/content/drive/MyDrive/{data_dir_name}"

import os

drive_path = "/content/drive"
from google.colab import drive

try:
    drive.mount(drive_path, force_remount=True)
    os.makedirs(data_dir, exist_ok=True)
except:
    print('Mount Error')
    raise

if os.path.exists(f"{data_dir}/script.pre.sh"):
    ! chmod +x {data_dir}/script.pre.sh
    ! {data_dir}/script.pre.sh

models_path = f"{data_dir}/models"
output_path = f"{data_dir}/outputs"
config_path = f"{data_dir}/config"
scripts_path = f"{data_dir}/scripts"
extensions_file_path = f"{data_dir}/extensions.txt"

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(config_path, exist_ok=True)
os.makedirs(scripts_path, exist_ok=True)
os.makedirs(f"{models_path}/Stable-diffusion", exist_ok=True)
os.makedirs(f"{models_path}/VAE", exist_ok=True)
os.makedirs(f"{models_path}/embeddings", exist_ok=True)
os.makedirs(f"{models_path}/hypernetworks", exist_ok=True)
os.makedirs(f"{models_path}/Lora", exist_ok=True)

for script in os.listdir(scripts_path):
    ! rm -Rf stable-diffusion-webui/scripts/{script} && ln -s {scripts_path}/{script} stable-diffusion-webui/scripts/{script}

for dir in os.listdir(models_path):
    if dir == "embeddings":
        ! rm -Rf stable-diffusion-webui/embeddings && ln -s {models_path}/embeddings stable-diffusion-webui/embeddings
    else:
        ! rm -Rf stable-diffusion-webui/models/{dir} && ln -s {models_path}/{dir} stable-diffusion-webui/models/{dir}

! rm -Rf stable-diffusion-webui/outputs && ln -s {data_dir}/outputs stable-diffusion-webui/outputs

for filename in ["config.json", "ui-config.json", "styles.csv", "artists.csv"]:
    ! rm -f stable-diffusion-webui/{filename}
    filepath = f"{config_path}/{filename}"
    if not os.path.exists(filepath):
        if filename.endswith(".json"):
            with open(filepath, mode="w") as f:
                f.write("{}")
        else:
            ! touch {config_path}/{filename}
    if filename == "styles.csv":
        ! cp {config_path}/styles.csv stable-diffusion-webui/styles.csv
    else:
        ! ln -s {config_path}/{filename} stable-diffusion-webui/{filename}

## 2 - 下載模型

In [ ]:
# @title 2.1 下載SD模型
# @markdown 需要下載SD模型組合包嗎？
need_to_download_sd = True # @param {type:"boolean"}
# @markdown 選擇SD模型組合包
model_package = "AnythingV4.5" # @param ["AnythingV3", "AnythingV4.5", "AbyssOrangeMix3", "CounterfeitV2.5", "PastalMix", "CamelliaMix_2.5D", "ChilloutMix_Ni_fix", "其他(下方填寫)"] 
# @markdown 其他SD模型下載網址
sd_url = "" #@param {type:"string"}
vae_url = "" #@param {type:"string"}
# @markdown ---
# @markdown - 除了利用Colab下載模型，推薦使用雲端硬碟捷徑以節省空間。
# @markdown  - [[AI繪圖] WebUI Colab玩家小技巧：使用雲端硬碟捷徑節省空間](https://home.gamer.com.tw/artwork.php?sn=5677784)
# @markdown - Anything, AbyssOrangeMix, Counterfeit 開啟 VAE 圖片色彩比較豐富
# @markdown  - SD VAE > anime.vae.pt
# @markdown - CamelliaMix_2.5D 開啟 VAE 圖片色彩比較豐富
# @markdown  - SD VAE > vae-ft-mse-840000-ema-pruned.safetensors
# @markdown - PastalMix 和 ChilloutMix 不需要設定 VAE
# @markdown  - SD VAE > Automatic

sd_urls = []
vae_urls = []
if model_package == "AnythingV3":
    sd_urls.append("https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp32-pruned.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/VAE/anime.vae.pt")
elif model_package == "AnythingV4.5":
    sd_urls.append("https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/VAE/anime.vae.pt")
elif model_package == "AbyssOrangeMix3":
    sd_urls.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3_orangemixs.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/VAE/anime.vae.pt")
elif model_package == "CounterfeitV2.5":
    sd_urls.append("https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_pruned.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/VAE/anime.vae.pt")
elif model_package == "PastalMix":
    sd_urls.append("https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix-better-vae-fp32.safetensors")
elif model_package == "CamelliaMix_2.5D":
    sd_urls.append("https://civitai.com/api/download/models/19779?type=Model&format=SafeTensor")
    vae_urls.append("https://civitai.com/api/download/models/19779?type=VAE&format=Other")
elif model_package == "ChilloutMix_Ni_fix":
    sd_urls.append("https://civitai.com/api/download/models/11745")
elif model_package == "其他(下方填寫)":
    sd_urls.append(sd_url)
    vae_urls.append(vae_url)

if sd_urls != [] and need_to_download_sd == True:
    %cd {models_path}/Stable-diffusion
    for sd_url in sd_urls:
        if sd_url != "":
            ! wget -nc --content-disposition {sd_url}

if vae_urls != [] and need_to_download_sd == True:
    %cd {models_path}/VAE
    for vae_url in vae_urls:
        if vae_url != "":  
            ! wget -nc --content-disposition {vae_url}

In [ ]:
# @title 2.2 下載 Embedding + Hypernetwork + LoRA

# @markdown Embedding
EasyNegative = True # @param {type:"boolean"}
embedding_urls = ""  # @param {type:"string"}

# @markdown Hypernetwork
hypernetwork_urls = ""  # @param {type:"string"}

# @markdown LoRA
koreanDollLikeness_v10 = False # @param {type:"boolean"}
koreanDollLikeness_v15 = False # @param {type:"boolean"}
taiwanDollLikeness_v10 = False # @param {type:"boolean"}
japaneseDollLikeness_v10 = False # @param {type:"boolean"}
lora_urls = ""  # @param {type:"string"}

# @markdown ---
# @markdown - [EasyNegative](https://civitai.com/models/7808/easynegative)
# @markdown   - 泛用負面 embedding
# @markdown   - 觸發詞為 EasyNegative, 加在負面 prompt
# @markdown - DollLikeness 系列
# @markdown   - 觸發詞為 woman, girl
# @markdown   - 模型權重建議為 0.5
# @markdown - 網址之間用英文逗號隔開
# @markdown - [LoRA預覽圖請自行添加，格式為：模型名稱.png](https://i.imgur.com/XooSV8p.png)

embeddings = []
embedding_urls = list(map(str.strip, embedding_urls.split(",")))
if EasyNegative == True:
    embeddings.append("https://civitai.com/api/download/models/9208")
if embedding_urls != ['']:
    embeddings = embeddings + embedding_urls

if embeddings != []:
    %cd {models_path}/embeddings
    for embedding in embeddings:
        ! wget -nc --content-disposition {embedding}

hypernetworks = []
hypernetwork_urls = list(map(str.strip, hypernetwork_urls.split(",")))
if hypernetwork_urls != ['']:
    hypernetworks = hypernetworks + hypernetwork_urls

if hypernetworks != []:
    %cd {models_path}/hypernetworks
    for hypernetwork in hypernetworks:
        ! wget -nc --content-disposition {hypernetwork}

loras = []
lora_urls = list(map(str.strip, lora_urls.split(",")))
if koreanDollLikeness_v10 == True:
    loras.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/LoRA/DollLikeness/koreanDollLikeness_v10.safetensors")
if koreanDollLikeness_v15 == True:
    loras.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/LoRA/DollLikeness/koreanDollLikeness_v15.safetensors")
if taiwanDollLikeness_v10 == True:
    loras.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/LoRA/DollLikeness/taiwanDollLikeness_v10.safetensors")
if japaneseDollLikeness_v10 == True:
    loras.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/LoRA/DollLikeness/japaneseDollLikeness_v10.safetensors")
if lora_urls != ['']:
    loras = loras + lora_urls

if loras != []:
    %cd {models_path}/Lora
    for lora in loras:
        ! wget -nc --content-disposition {lora}

## 3 - Extensions

In [ ]:
# @title 3.1 導入 Extensions
load_extensions = False # @param {type:"boolean"}

%cd /content
extensions_path = f"{data_dir}/extensions"
os.makedirs(extensions_path, exist_ok=True)

if load_extensions == True:
    ! rm -Rf stable-diffusion-webui/extensions && cp -r {extensions_path} stable-diffusion-webui/

In [ ]:
# @title 3.2 下載&更新 Extensions
localization_zh_TW = True # @param {type:"boolean"}
tag_complete = True # @param {type:"boolean"}
image_browser = True # @param {type:"boolean"}
Civitai_Helper = True # @param {type:"boolean"}
locon = True # @param {type:"boolean"}
control_net = False # @param {type:"boolean"}
openpose_editor = False # @param {type:"boolean"}
posex = False # @param {type:"boolean"}
depth_lib = False # @param {type:"boolean"}
latent_couple = False # @param {type:"boolean"}
composable_lora = False # @param {type:"boolean"}
cutoff = False # @param {type:"boolean"}
extension_urls = ""  # @param {type:"string"}

# @markdown ---
# @markdown - [localization_zh_TW](https://github.com/benlisquare/stable-diffusion-webui-localization-zh_TW)：繁體中文翻譯介面
# @markdown - [tag_complete](https://github.com/DominikDoom/a1111-sd-webui-tagcomplete)：Danbooru tag 建議
# @markdown - [image_browser](https://github.com/AlUlkesh/stable-diffusion-webui-images-browser)：生成圖片預覽
# @markdown - [Civitai_Helper](https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper)：Civitai 模型管理工具
# @markdown - [locon](https://github.com/KohakuBlueleaf/a1111-sd-webui-locon)：使用 LoCon 模型
# @markdown - [control_net](https://github.com/Mikubill/sd-webui-controlnet)：多種控制姿勢構圖方法
# @markdown - [openpose_editor](https://github.com/fkunn1326/openpose-editor)：繪製調整 openpose 骨架
# @markdown - [posex](https://github.com/hnmr293/posex)：繪製調整 openpose 骨架
# @markdown - [depth_lib](https://github.com/jexom/sd-webui-depth-lib)：編輯手勢 depth 圖
# @markdown - [latent_couple](https://github.com/ashen-sensored/stable-diffusion-webui-two-shot)：分割繪製多個主題或人物
# @markdown - [composable_lora](https://github.com/opparco/stable-diffusion-webui-composable-lora)：限制 LoRA 作用範圍
# @markdown - [cutoff](https://github.com/hnmr293/sd-webui-cutoff)：限制 token 作用範圍（通常是顏色）
# @markdown - 網址之間用英文逗號隔開<br>

extensions = []
extension_urls = list(map(str.strip, extension_urls.split(",")))

if localization_zh_TW == True:
    extensions.append("https://github.com/benlisquare/stable-diffusion-webui-localization-zh_TW")
if tag_complete == True:
    extensions.append("https://github.com/DominikDoom/a1111-sd-webui-tagcomplete")
if image_browser == True:
    extensions.append("https://github.com/AlUlkesh/stable-diffusion-webui-images-browser")
if Civitai_Helper == True:
    extensions.append("https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper")
if locon == True:
    extensions.append("https://github.com/KohakuBlueleaf/a1111-sd-webui-locon")
if control_net == True:
    extensions.append("https://github.com/Mikubill/sd-webui-controlnet")
if openpose_editor == True:
    extensions.append("https://github.com/fkunn1326/openpose-editor")
if posex == True:
    extensions.append("https://github.com/hnmr293/posex")
if depth_lib == True:
    extensions.append("https://github.com/jexom/sd-webui-depth-lib")
if latent_couple == True:
    extensions.append("https://github.com/ashen-sensored/stable-diffusion-webui-two-shot")
if composable_lora == True:
    extensions.append("https://github.com/opparco/stable-diffusion-webui-composable-lora")
if cutoff == True:
    extensions.append("https://github.com/hnmr293/sd-webui-cutoff")
if extension_urls != ['']:
    extensions = extensions + extension_urls

import os
import json

if extensions != []:
    %cd /content/stable-diffusion-webui/extensions
    for extension in extensions:
        extension_name, _ = os.path.splitext(extension.split("/")[-1])
        if not os.path.isdir(extension_name):
            ! git clone {extension}
        else:
            %cd {extension_name}
            ! git pull
            %cd ..

def read_config(filename):
  with open(filename, "r") as f:
    config = json.load(f)
  return config

def write_config(filename, config):
  with open(filename, "w") as f:
    json.dump(config, f, indent=4)

%cd /content/stable-diffusion-webui/extensions
config_file = f"{config_path}/config.json"
config = read_config(config_file)
if "CLIP_stop_at_last_layers" not in config or config["CLIP_stop_at_last_layers"] == 1:
    config["CLIP_stop_at_last_layers"] = 2
if "quicksettings" not in config or config["quicksettings"] == "sd_model_checkpoint":
    config["quicksettings"] = "sd_model_checkpoint, sd_vae"    
if os.path.exists("sd-webui-controlnet") and ("control_net_max_models_num" not in config or config["control_net_max_models_num"] == 1): 
    config["control_net_max_models_num"] = 3
if os.path.exists("stable-diffusion-webui-localization-zh_TW") and ("localization" not in config or config["localization"] == "None"): 
    config["localization"] = "zh_TW"
write_config(config_file, config)  

In [ ]:
# @title 3.3 下載 ControlNet 模型
# @markdown [ControlNet](https://github.com/lllyasviel/ControlNet)
control_canny = False # @param {type:"boolean"}
control_depth = False # @param {type:"boolean"}
control_hed = False # @param {type:"boolean"}
control_mlsd = False # @param {type:"boolean"}
control_normal = False # @param {type:"boolean"}
control_openpose = True # @param {type:"boolean"}
control_scribble = False # @param {type:"boolean"}
control_seg = False # @param {type:"boolean"}
# @markdown [T2I Adapter](https://github.com/TencentARC/T2I-Adapter)
t2iadapter_canny = False # @param {type:"boolean"}
t2iadapter_color = False # @param {type:"boolean"}
t2iadapter_depth = False # @param {type:"boolean"}
t2iadapter_keypose = False # @param {type:"boolean"}
t2iadapter_openpose = False # @param {type:"boolean"}
t2iadapter_seg = False # @param {type:"boolean"}
t2iadapter_sketch = False # @param {type:"boolean"}
t2iadapter_style = False # @param {type:"boolean"}
# @markdown ---
# @markdown - 多重 ControlNet 請到 Settings > ControlNet > Multi ControlNet 設置
# @markdown - 參考資料
# @markdown  - https://github.com/Mikubill/sd-webui-controlnet
# @markdown  - [Stable diffusion ControlNet使用心得](https://home.gamer.com.tw/artwork.php?sn=5662905)
# @markdown  - [Avatarposemaker](https://avatarposemaker.deezein.com/)
# @markdown  - [Open-pose-editor](https://github.com/ZhUyU1997/open-pose-editor)
# @markdown  - [Posemaniacs](https://www.posemaniacs.com/)

import os

control_net_models = []
t2iadapter_models = []

if control_canny == True:
    control_net_models.append("canny")
if control_depth == True:
    control_net_models.append("depth")
if control_hed == True:
    control_net_models.append("hed")
if control_mlsd == True:
    control_net_models.append("mlsd")
if control_normal == True:
    control_net_models.append("normal")
if control_openpose == True:
    control_net_models.append("openpose")
if control_scribble == True:
    control_net_models.append("scribble")
if control_seg == True:
    control_net_models.append("seg")

if t2iadapter_canny == True:
    t2iadapter_models.append("canny")
if t2iadapter_color == True:
    t2iadapter_models.append("color")
if t2iadapter_depth == True:
    t2iadapter_models.append("depth")
if t2iadapter_keypose == True:
    t2iadapter_models.append("keypose")
if t2iadapter_openpose == True:
    t2iadapter_models.append("openpose")
if t2iadapter_seg == True:
    t2iadapter_models.append("seg")
if t2iadapter_sketch == True:
    t2iadapter_models.append("sketch")
if t2iadapter_style == True:
    t2iadapter_models.append("style")

%cd /content/stable-diffusion-webui/extensions

if os.path.exists('sd-webui-controlnet'):
    %cd sd-webui-controlnet/models
    for control_net_model in control_net_models:
        ! wget -nc --content-disposition https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_{control_net_model}-fp16.safetensors
    for t2iadapter_model in t2iadapter_models:
        ! wget -nc --content-disposition https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_{t2iadapter_model}_sd14v1.pth        

In [ ]:
# @title 3.4 儲存 Extensions
save_extensions = False # @param {type:"boolean"}

%cd /content
if save_extensions == True:   
    ! cp -r stable-diffusion-webui/extensions {data_dir}

## 4 - 啟動 WebUI

In [ ]:
# @title 4.1 命令行參數
ngrok = False # @param {type:"boolean"}
your_authtoken = "" # @param {type:"string"}
# @markdown ---
# @markdown 如果 public URL 有問題可以嘗試改用 ngrok
# @markdown - 註冊 [ngrok](https://ngrok.com/) 帳號
# @markdown - Your Authtoken > Copy
# @markdown - 勾選 ngrok > 貼上 authtoken
# @markdown - 等 local URL 出來之後再點 ngrok URL

import os

os.environ["COMMANDLINE_ARGS"] = "--no-half-vae --xformers --gradio-queue --lowram \
                  --no-hashing --enable-insecure-extension-access \
                  --disable-safe-unpickle --opt-channelslast --theme dark"

if ngrok == True:
    os.environ["COMMANDLINE_ARGS"] += (" --ngrok " + your_authtoken)
else:
    os.environ["COMMANDLINE_ARGS"] += " --share"

In [ ]:
# @title 4.2 啟動 WebUI
# @markdown - 點選 public URL 開啟 WebUI<br>
# @markdown ![](https://i.imgur.com/y3xGiIX.png)

%cd /content/stable-diffusion-webui/
! python launch.py

In [ ]:
# @title 4.3 儲存 styles.csv
! cp /content/stable-diffusion-webui/styles.csv {config_path}/styles.csv